In [ ]:
include("solver_second.jl")
include("kneser_ney.jl")

In [ ]:
filePath = "data/seq_foursq.txt";
splitRatio = 0.6;
totalChevNodes = 15;
gridSize = 0.01;

numDic1, numDic2, maxId, splitVec = init_dict_second(filePath, splitRatio);
Q_rhomp, P_rhomp, alpha = overall_solve(numDic1, maxId, totalChevNodes, gridSize);
~, P_first, likelihood_first = one_solve(numDic1, maxId, 0);
P_second, indDic1 = init_matrix_second(numDic1);
println(size(P_rhomp))
println(size(P_second))


In [ ]:
K = [1,2,3,4,5]
x_1 = mrr_first(numDic2, P_first)
y_1 = precision_first(numDic2, P_first, K)

x_2 = mrr_second(numDic2, P_second, indDic1, maxId+1)
y_2 = precision_second(numDic2, P_second, K, indDic1);

x_rhomp = mrr_rhomp(numDic2, Q_rhomp, P_rhomp, alpha)
y_rhomp = precision_rhomp(numDic2, Q_rhomp, P_rhomp, alpha, K)

println("MRR for 1-st MC, 2-st MC, rhomp are:")
println([x_1;x_2;x_rhomp])
println("Precisions for 1-st MC, 2-st MC, rhomp are:")
println(y_1); println(y_2); println(y_rhomp)

In [ ]:
numDic1_kn, numDic2_kn, dist_kn, D_kn = init_kneser_ney(filePath, splitRatio);
P_first_kn = first_kneser_ney(numDic1_kn, dist_kn, D_kn);
indDic_kn, gama_kn, uniq_kn, totalCount_kn = second_kneser_ney(numDic1_kn, numDic2_kn, P_first_kn, D_kn);

x_kn_1 = mrr_first_kneser_ney(numDic2_kn, P_first_kn);
y_kn_1 = precision_first_kneser_ney(numDic2_kn, P_first_kn, [1,3,5,7,10]);
x_kn_2 = mrr_second_kneser_ney(numDic2_kn, P_first_kn, D_kn, indDic_kn, gama_kn, uniq_kn, totalCount_kn);
y_kn_2 = precision_second_kneser_ney(numDic2_kn, P_first_kn, D_kn, indDic_kn, gama_kn, uniq_kn, totalCount_kn, K);

println("MRR for 1-st Kneser Ney and 2-nd Kneser Ney are:")
println([x_kn_1;x_kn_2])
println("Precisions for 1-st Kneser Ney and 2-nd Kneser Ney are:")
println(y_kn_1); println(y_kn_2)

In [ ]:
include("ritf.jl")

compressRatio = 0.05;
mu = 0; std = 0.01; a = 0.05; lambda = 5e-5; maxiter = 10000000;

inputData, cate, dict, numDic1, numDic2, maxId = init_ritf(filePath, splitRatio);
dim = round(Int, maxId*compressRatio);
U, I, TU, TI = PITF(inputData, cate, dict, maxId, mu, std, a, lambda, dim, maxiter);

rankDic = compute_rank(U, I, TU, TI, numDic2);
x_ritf = mrr_ritf(numDic2, rankDic);
y_ritf = precision_ritf(numDic2, rankDic, K);

println("MRR for RITF is:")
println(x_ritf)
println("Precisions for RITF are:")
println(y_ritf)

In [ ]:
# the raw results for computing the precisions for different groups of states
k = 3;

countArray = compute_count(numDic1, maxId);
occ_1,z_1 = pr_piece_first(countArray, numDic2, P_first, k);
occ_2,z_2 = pr_piece_second(countArray, numDic2, P_second, indDic1, k)
occ_rhomp, z_rhomp = pr_piece_rhomp(countArray, numDic2, Q_rhomp, P_rhomp, alpha, k)

countArray_kn = compute_count(numDic1_kn, size(P_first_kn,1));
occ_kn_1, z_kn_1 = pr_piece_first_kneser_ney(countArray_kn, numDic2_kn, P_first_kn, 3);
occ_kn_2, z_kn_2 = pr_piece_second_kneser_ney(countArray_kn, numDic2_kn, P_first_kn, D_kn, indDic_kn, gama_kn, uniq_kn, totalCount_kn, 3);

In [ ]:
# results for order >= 3 for RHOMP
include("solver_high.jl")
order = 4;

temp = []; push!(temp, P_rhomp); push!(temp, Q_rhomp);
Prob_rhomp = []; push!(Prob_rhomp, temp);
Prob_rhomp = high_solver_rhomp(alpha, maxId, Prob_rhomp, splitVec, order, filePath)

evaluate_high_rhomp(alpha, maxId, Prob_rhomp, splitVec, filePath, K)


In [ ]:
# results for order >= 3 for MC
order = 4;

Prob_mc = []; dicts = [];
push!(Prob_mc, P_second); push!(dicts, indDic1);
Prob_mc = high_solver_mc(Prob_mc, dicts, splitVec, order, filePath);

evaluate_high_mc(maxId, Prob_mc, dicts, splitVec, filePath, K)

In [ ]:
# results for order >= 3 for Kneser Ney
order = 4;

numDic1_kn, numDic2_kn, Prob_kn, dicts_kn, gamas_kn, dist_kn, P_first_kn, gama_kn, splitVec_kn = high_slover_kn(filePath, splitRatio, order);
evaluate_highkneser(Prob_kn, dicts_kn, gamas_kn, dist_kn, P_first_kn, gama_kn, splitVec_kn, filePath, K)

In [ ]:
# For the implementation of the LME methods please refer to: http://www.cs.cornell.edu/~shuochen/lme/multilme_page.html
# The codes in LME.jl processes the output from the above software and evaluate the MRR and precisions